# Chapter7
## Model Performance and Validation

In [2]:
import os, sys
import numpy as np
from math import floor, ceil
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.patches import Patch, Rectangle
from matplotlib.lines import Line2D
from tqdm import tqdm
import h5py
# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))
from sample import Sample
from dataset import Dataset
from database import Database
from dataset_simulation import DatasetSimulation
from cfar_detector import CFARDetector
from nn_detector import NeuralNetworkDetector
from yolo_detector import YOLODetector
from cfar_detector_generator import CFARDetectorGenerator


dataset_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'awdEvents1')
database_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','databases', 'awdEvents1')
result_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','results', 'awdEvents1')
dataset_sim_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'simulations', 'simple' ,'whistler')
site = 'sanae'
t_res, f_res = 0.006395061728395062, 0.15503875968992248
database = Database(dataset_loc, database_loc, site)
cfar_detector_gen = CFARDetectorGenerator(dataset_loc, database_loc, dataset_sim_loc, result_loc, site, t_res, f_res)

linewidth = 2
font_title = 32
font_legend = 20
font_label = 24
font_ticks = 18
font = [font_title, font_legend, font_label, font_ticks, linewidth] # [None]*5

### 7.3.1 Cross-Correlation using Whistler Kernel (CCWK)

## SANAE IV

In [3]:
site = 'sanae'
t_res, f_res = 0.012781818181818182, 0.07751937984496124
database = Database(dataset_loc, database_loc, site)
dataset_sim = DatasetSimulation(dataset_sim_loc, t_res, f_res)
cfar_detector_gen = CFARDetectorGenerator(dataset_loc, database_loc, dataset_sim_loc, result_loc, site, t_res, f_res)

linewidth = 2
font_title = 32
font_legend = 20
font_label = 24
font_ticks = 18
font = [font_title, font_legend, font_label, font_ticks, linewidth] # [None]*5

files = database.get_train()
file = np.random.choice(files)

In [4]:
#CCSW
t, t_params = 'zscore',None
n = None
f_min, f_max = 1.5, 9.5
N,G,k,Ts,Tl,X_dB = 12,10,13,5,3,0.5
train = True
cfar_detector_gen = CFARDetectorGenerator(dataset_loc, database_loc, dataset_sim_loc, result_loc, site, t_res, f_res)
cfar_detector_gen.dataset_cross_correlation_gen(transforms=t, transforms_params=t_params, f_min=f_min, f_max=f_max, 
                                               An=0.35, D0=35, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                               train=train, n=n,force=False)
data = cfar_detector_gen.dataset_cross_correlation_load(transforms=t, transforms_params=t_params, f_min=f_min, f_max=f_max, 
                                               An=0.35, D0=35, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                               train=train, n=n)
confusion_matrix = data['confusion matrix']
performance = cfar_detector_gen.performance(confusion_matrix)
print(confusion_matrix)
print(performance)

100%|██████████| 373/373 [00:56<00:00,  6.57it/s]


[1121, 1314, 197, 0]
{'recall': 0.851, 'precision': 0.46, 'f1 score': 0.597, 'g measure': 0.626, 'false alarm': 0.54, 'misdetection': 0.149}


In [2]:
n = None
f_range = [[1.5,9.5],[1.5,9.5],[1.5,9.5],[1.5,9.5],]
params = [[10,5,13,5,7,0.3],[12,10,13,5,3,0.5],[8,5,5,0,7,0.4],[12,10,13,8,5,0.4]]
transforms = [None, 'zscore', 'detrend', 'detrend']
transforms_params = [None, None, 'constant', 'linear']
train = True

for i in range(len(params)):
    t,t_params = transforms[i], transforms_params[i]
    f_min, f_max = f_range[i]
    N,G,k,Ts,Tl,X_dB = params[i]
    print((t,t_params))
    cfar_detector_gen.dataset_cross_correlation_gen(transforms=None, transforms_params=None, f_min=f_min, f_max=f_max, 
                                                   An=0.35, D0=80, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                                   train=train, n=n)
    data = cfar_detector_gen.dataset_cross_correlation_load(transforms=None, transforms_params=None, f_min=f_min, f_max=f_max, 
                                                   An=0.35, D0=80, magnitude=1, N=N, G=G, k=k, Ts=Ts, Tl=Tl, X_dB=X_dB,
                                                   train=train, n=n)
    confusion_matrix = data['confusion matrix']
    performance = cfar_detector_gen.performance(confusion_matrix)
    print(confusion_matrix)
    print(performance)

(None, None)
(None,None)_(1.5,9.5)_(0.35,80,1)_(10,5,13,5,7,0.3)_(True,None,1).corr already exists
[118, 22, 1200, 0]
{'recall': 0.09, 'precision': 0.843, 'f1 score': 0.162, 'g measure': 0.275, 'false alarm': 0.157, 'misdetection': 0.91}
('zscore', None)


100%|██████████| 373/373 [01:06<00:00,  5.63it/s]

[27, 3, 1291, 0]
{'recall': 0.02, 'precision': 0.9, 'f1 score': 0.04, 'g measure': 0.136, 'false alarm': 0.1, 'misdetection': 0.98}
('detrend', 'constant')



100%|██████████| 373/373 [01:09<00:00,  5.35it/s]


[130, 11, 1188, 0]
{'recall': 0.099, 'precision': 0.922, 'f1 score': 0.178, 'g measure': 0.302, 'false alarm': 0.078, 'misdetection': 0.901}
('detrend', 'linear')


100%|██████████| 373/373 [01:11<00:00,  5.22it/s]


[60, 14, 1258, 0]
{'recall': 0.046, 'precision': 0.811, 'f1 score': 0.086, 'g measure': 0.192, 'false alarm': 0.189, 'misdetection': 0.954}
